Group Name: AG xx.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

In [1]:
from __future__ import annotations
import csv
import math
import random
from itertools import combinations
from typing import Tuple, List, Dict
from heapq import heappush, heappop

# Question 2

## Introduction to question 2

In the second question of this assignment, we will explore the use of local search in genome assembly.

We will use local search to assemble (construct) a large part of the nucleotide sequence of the monkeypox virus, which has been downloaded from the National Center for Biotechnology Information in the United States. Please note that no additional or specialized knowledge of biology or bioinformatics is required for this assignment. (Actually, the technical specifics of bioinformatics have been adapted and simplified for the purposes of this computer science assignment, so if you are a biologist, please do not apply preexisting knowledge to solve the problem. Furthermore, you should not attempt to search up the genome on genomic databases to "guess" the actual sequence, since we are more interested in your coding methodology rather than your attempts at reproducing a known sequence.)

This is an introductory computer science assignment and not a bioinformatics assignment; we are simply using bioinformatics as a use case to illustrate the applicability of local search to the natural sciences. Therefore, no knowledge of bioinformatics is assumed or required. In the paragraphs that follow, I will give a short crash course which will cover all the domain knowledge you will need to know in order to tackle this problem.  

For technical reasons, when we analyze the nucleotide sequence (genome) of a virus, we usually cannot “read” it in one fell swoop. We have to read the genome in parts, because the genome is usually too long for the machine to read in a single sitting. To simplify things, a “read” is a single view of part of the genome; think of it as a SUBSTRING, a partial view of the whole genome. After we have generated multiple reads of a genome, we then have to “stitch”, or combine, the different reads of the genome together. This process of stitching up reads of a genome into the final sequence is known as genome assembly. However, the different reads of the genome cannot just be concatenated like usual string concatenation. It’s not a situation where you have one read, “Hello”, and another read, “World”, and all you need to do is concatenate both strings together to make “Hello World”. Among other reasons, there are two major reasons why you can’t do so:

1. You do not know which read came first. The reads are not ordered. How do you know “Hello” came after “World”? The answer is that you don’t. Imagine how complicated this situation might be if you had more than two reads. (This is indeed our situation, where we have $n$ reads, and $n>>2$.)

2. One read may contain a substring contained in another read. Specifically, without loss of generality, part of the ending $x$ characters of a read (i.e., suffix) might also be found in the starting $x$ positions (i.e., prefix) of another read.

- A computer scientist usually creates opportunities from problems. While this may be a “problem” in that you just can’t concatenate two strings blindly, the fact that strings contain shared “substrings” is actually a very helpful clue that you can use to “join” strings together. 

- Note that the choice of the value of $x$ could be a hyperparameter decided by the computer scientist.

## Your tasks

In this part of the assignment, you will work with (simulated) reads that I have generated from the nucleotide sequence of the monkeypox virus. In reality, bioinformatics is far more complicated, but here we will work with a simplified situation. Your task is to examine the reads that I have provided for you, and from there “infer” the nucleotide sequence that might have produced those reads. 

The reads are provided in the csv file `data.csv` which simply provides a list of unique strings. Note that you should NOT assume any particular ordering of the strings in this dataframe. In fact, the strings have already been shuffled randomly. 

NOTE: You are not allowed to use `pandas` or any other libraries apart from the Python STL to load the csv file.

### Task A (3 marks): 

Create a directed graph. The nodes in the graph are the strings in the list of reads. An edge should be drawn FROM read A TO read B if and only if a suffix (of length $x$) of read A is also a prefix (obviously, also of length $x$) of read B. For the purposes of the assignment, limit the value of $x$ to between 5 and 30, both inclusive. That is, to be clear, $5\leq x\leq 30$. The weight of an edge between read A and read B should be the NEGATED value of $x$, i.e. $-x$. 

In your Jupyter notebook, please report the number of edges in your graph. Provide a barplot or histogram which shows the number of edges with different weights or weight categories. In this task, you are free to use plotting libraries such as `matplotlib` or `seaborn` to plot this graph.

As an example, if read A is "TACTAGT" and read B is "TAGTCCCCT", then an edge is drawn FROM read A TO read B (i.e., $A \rightarrow B$) with weight of $-4$. This is because the 4-suffix "TAGT" is also the 4-prefix of read B; in other words, the last 4 characters of read A (a substring of length 4) overlap with the first 4 characters of read B (a substring of length 4).

### Task B (7 marks): 

From Task A, you now have a graph which shows connections between reads based on how they overlap, in theory you could draw a path through the graph and thereby derive the full sequence (genome).

Task B asks you to use local search method(s) to determine a path through this directed graph of strings. 

- You are expected to use simulated annealing and tune the relevant configuration settings and hyperparameters. The minimum requirement is to implement simulated annealing.

- Explain tha rationale behind the choice of scheduling strategy and parameters.

- However, you may also explore other search methods in addition to simulated annealing. Marks will be awarded for effort.

Note the following constraints:

1. The path has to go through each and every vertex exactly once. For computer scientists, this constraint is reminiscent of the "Traveling Salesman's Problem", except that unlike TSP, we should not need to go back to the starting vertex again. 

2. For the purposes of neighbor generation / action selection at each node, bear in mind that a path through the graph which minimizes the total number of nucleotides in the assembled sequence is the preferred path. To state that another way, the assembled sequence should be derived from a path that goes through EACH and EVERY vertex exactly once, however we want this assembled sequence to be AS SHORT AS POSSIBLE.

3. You are not given the starting (source/origin) or ending (destination) vertex.

4. For avoidance of ambiguity, no cycles are allowed. You must not visit a vertex more than once.

5. You are not allowed to use any libraries apart from the Python Standard Library.
No import statements which import libraries outside of the Python STL should be found within your answer for Task B.

Please remember to report the assembled sequence that you obtain. Although it would be great if you can come up with a good sequence, please feel reassured that we are more interested in your APPROACH to the problem, and so you can potentially get a reasonable score on this task even if your solution is "wrong". It is the process, rather than the result, which matters more.

In [2]:
def calculate_weight(a_string: str, b_string: str) -> int:
    """
    Args:
        a_string (str): String to compare as from node
        b_string (str): String to compare as to node
    
    Returns:
        int:
            Weight with -ve of a -> b
    """
    if a_string == b_string:
        return -30
    for i in range(1, 25):
        if a_string[i:] == b_string[:-i]:
            return -(30-i)
    return 0

In [3]:
def calculate_weights(csv_data:dict) -> Dict[Tuple[str, str], int]:
    """
    Args:
        csv_data (dict):
            Data from csv with the following schema {READ_NAME (str): READ_SEQUENCE (str)}
    
    Returns:
        Dict[Tuple[str, str], int]:
            Dict with the following schema:  
            {('READ_#', 'READ_#2'): WEIGHT_OF_READ#_TO_READ#2}
    """
    weights = {}
    set_added = set()
    for sequences in combinations(data.items(), 2):
        ((read_name_1, sequence_1), (read_name_2, sequence_2)) = sequences
        if read_name_1 == read_name_2:
            continue
        
        weight_1_2 = calculate_weight(sequence_1[-30:],sequence_2[:30])
        if weight_1_2:
            weight_lib = weights.get(read_name_1, [])
            heappush(weight_lib, (weight_1_2, read_name_2))
            weights[read_name_1] = weight_lib

        weight_2_1 = calculate_weight(sequence_2[-30:],sequence_1[:30])
        if weight_2_1:
            weight_lib = weights.get(read_name_2, [])
            heappush(weight_lib, (weight_2_1, read_name_1))
            weights[read_name_2] = weight_lib
            
    return weights

def load_data(file) -> dict:
    data = {}
    with open(file) as file:
        entries = map(dict, csv.DictReader(file))
        for entry in entries:
            data[entry['read_index']] = entry['read_sequence']
    return data

In [4]:
data = load_data('data.csv')

In [5]:
import copy
backup = copy.deepcopy(data)

In [6]:
weights = calculate_weights(data)

In [8]:
weights

{'read_0': [(-16, 'read_224')],
 'read_566': [(-24, 'read_0'), (-6, 'read_294')],
 'read_69': [(-25, 'read_442'),
  (-6, 'read_234'),
  (-6, 'read_1'),
  (-6, 'read_444')],
 'read_114': [(-22, 'read_154'),
  (-6, 'read_1'),
  (-6, 'read_234'),
  (-6, 'read_444')],
 'read_1': [(-18, 'read_228'), (-6, 'read_344')],
 'read_369': [(-26, 'read_1')],
 'read_2': [(-22, 'read_223')],
 'read_390': [(-18, 'read_2')],
 'read_3': [(-24, 'read_355')],
 'read_554': [(-22, 'read_3'), (-7, 'read_315')],
 'read_23': [(-20, 'read_4')],
 'read_4': [(-17, 'read_436')],
 'read_5': [(-15, 'read_185')],
 'read_241': [(-25, 'read_5')],
 'read_6': [(-12, 'read_433'), (-7, 'read_22'), (-7, 'read_485')],
 'read_194': [(-18, 'read_152'),
  (-7, 'read_373'),
  (-7, 'read_6'),
  (-6, 'read_150')],
 'read_462': [(-18, 'read_6'), (-6, 'read_289')],
 'read_7': [(-25, 'read_231')],
 'read_325': [(-14, 'read_7'),
  (-6, 'read_28'),
  (-6, 'read_326'),
  (-6, 'read_517')],
 'read_8': [(-23, 'read_590'), (-6, 'read_256')]

In [9]:
# Problem Class
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [10]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0, **kwds):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost, **kwds)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 
    def __eq__(self, other): return isinstance(other, Node) and self.state == other.state
    def expand_node(self, state, cost):
        new_node = Node(state, 
                   path_cost=self.path_cost + cost,
                   cost = cost,
                   parent=self)
        return new_node

In [11]:
# Code to generate neighbours, value of states, etc.
class TSP(Problem):
    #Implement TSP class here
    def calculate_weight(node: Node):
        pass
    
    def is_cycle(self, node: Node):
        read = node.state
        while node.parent is not None:
            if node.parent.state == read:
                return True
            node = node.parent
        return False
    
    def load_data(self) -> dict:
        """Loads data from path provided

        Args:
            path (str): local path to file

        Returns:
            dict: 
                dict in the following schema:
                {READ_INDEX (str): READ_SEQUENCE (str)}
        """
        data = {}
        with open(self.data_path) as file:
            entries = map(dict, csv.DictReader(file))
            for entry in entries:
                data[entry['read_index']] = entry['read_sequence']
        self.data = data

In [12]:
read = Node('read_390')

In [13]:
tsp = TSP(initial=read, data_path='data.csv', data=None)

In [14]:
tsp.load_data()

In [ ]:
tsp.data

In [15]:
def is_cycle(node: Node):
    read = node.state
    while node.parent is not None:
        if node.parent.state == read:
            return True
        node = node.parent
    return False

In [16]:
def get_node_index(read: str) -> int:
    """
    Get read index given read name
    """
    return int(read.split('_')[1])

def convert_node_index(read_index: int) -> str:
    """
    Get read index given read name
    """
    return f'read_{read_index}'

In [17]:
def find_lowest_path(node:Node):
    """
    Initial lowest path algo
    """
    visited = [0 for i in range(len(weights))]
    visited[get_node_index(node.state)] = 1
    current_node = (0, node)
    
    total_number_of_nodes = len(weights)
    travelled_nodes = 1
    while travelled_nodes < total_number_of_nodes:
        path_cost, read = current_node
        print('travelled ', travelled_nodes, 'read ', read)
        neighbours = weights[read.state]
        found_neighbours = False
        while neighbours:
            cost, neighbour = heappop(neighbours)
            neighbour_index = get_node_index(neighbour)
            if not visited[neighbour_index]:
                print('visited', neighbour)
                current_node = (cost, read.expand_node(neighbour, cost))
                travelled_nodes += 1
                visited[neighbour_index] = 1
                found_neighbours = True
                break
        if not found_neighbours:
            print(f'no neighbours, finding another node from {read.state}...')
            not_visited_index = visited.index(0)
            print(f'found index {not_visited_index}')
            current_node = (0, read.expand_node(convert_node_index(not_visited_index), 0))
            travelled_nodes += 1
            visited[not_visited_index] = 1
    print(travelled_nodes)
    return current_node

In [18]:
import copy
data = load_data('data.csv')
weights = calculate_weights(data)
backup = copy.deepcopy(weights)

In [19]:
cost, node = find_lowest_path(Node(state=f'read_596'))

travelled  1 read  <read_596>
visited read_68
travelled  2 read  <read_68>
visited read_403
travelled  3 read  <read_403>
visited read_471
travelled  4 read  <read_471>
visited read_490
travelled  5 read  <read_490>
visited read_188
travelled  6 read  <read_188>
visited read_507
travelled  7 read  <read_507>
visited read_142
travelled  8 read  <read_142>
visited read_378
travelled  9 read  <read_378>
visited read_449
travelled  10 read  <read_449>
visited read_530
travelled  11 read  <read_530>
visited read_176
travelled  12 read  <read_176>
visited read_169
travelled  13 read  <read_169>
visited read_457
travelled  14 read  <read_457>
visited read_27
travelled  15 read  <read_27>
visited read_523
travelled  16 read  <read_523>
visited read_358
travelled  17 read  <read_358>
visited read_129
travelled  18 read  <read_129>
visited read_380
travelled  19 read  <read_380>
visited read_65
travelled  20 read  <read_65>
visited read_183
travelled  21 read  <read_183>
visited read_39
travelle

travelled  398 read  <read_424>
visited read_167
travelled  399 read  <read_167>
visited read_76
travelled  400 read  <read_76>
visited read_187
travelled  401 read  <read_187>
visited read_126
travelled  402 read  <read_126>
visited read_327
travelled  403 read  <read_327>
visited read_359
travelled  404 read  <read_359>
visited read_128
travelled  405 read  <read_128>
visited read_539
travelled  406 read  <read_539>
visited read_538
travelled  407 read  <read_538>
visited read_164
travelled  408 read  <read_164>
visited read_466
travelled  409 read  <read_466>
visited read_64
travelled  410 read  <read_64>
visited read_546
travelled  411 read  <read_546>
visited read_570
travelled  412 read  <read_570>
visited read_211
travelled  413 read  <read_211>
visited read_544
travelled  414 read  <read_544>
visited read_467
travelled  415 read  <read_467>
visited read_210
travelled  416 read  <read_210>
visited read_251
travelled  417 read  <read_251>
visited read_425
travelled  418 read  <re

In [20]:
node.path_cost

-12011

In [ ]:
for i in range(0, len(data)):
    weights = copy.deepcopy(backup)
    try:
        (path_cost, node)= find_lowest_path(Node(state=f'read_{i}')) # change this and check the final number. Please confirm this number
    except:
        print(i)

Found node 596 and 68 with highest -ve costs based on greedy search

In [21]:
weights = copy.deepcopy(backup)
cost, node = find_lowest_path(Node(state=f'read_68'))

travelled  1 read  <read_68>
visited read_403
travelled  2 read  <read_403>
visited read_471
travelled  3 read  <read_471>
visited read_490
travelled  4 read  <read_490>
visited read_188
travelled  5 read  <read_188>
visited read_507
travelled  6 read  <read_507>
visited read_142
travelled  7 read  <read_142>
visited read_378
travelled  8 read  <read_378>
visited read_449
travelled  9 read  <read_449>
visited read_530
travelled  10 read  <read_530>
visited read_176
travelled  11 read  <read_176>
visited read_169
travelled  12 read  <read_169>
visited read_457
travelled  13 read  <read_457>
visited read_27
travelled  14 read  <read_27>
visited read_523
travelled  15 read  <read_523>
visited read_358
travelled  16 read  <read_358>
visited read_129
travelled  17 read  <read_129>
visited read_380
travelled  18 read  <read_380>
visited read_65
travelled  19 read  <read_65>
visited read_183
travelled  20 read  <read_183>
visited read_39
travelled  21 read  <read_39>
visited read_108
travelle

travelled  464 read  <read_390>
visited read_2
travelled  465 read  <read_2>
visited read_223
travelled  466 read  <read_223>
visited read_33
travelled  467 read  <read_33>
visited read_485
travelled  468 read  <read_485>
visited read_73
travelled  469 read  <read_73>
visited read_244
travelled  470 read  <read_244>
visited read_202
travelled  471 read  <read_202>
visited read_342
travelled  472 read  <read_342>
visited read_66
travelled  473 read  <read_66>
visited read_409
travelled  474 read  <read_409>
visited read_483
travelled  475 read  <read_483>
visited read_201
travelled  476 read  <read_201>
visited read_281
travelled  477 read  <read_281>
visited read_391
travelled  478 read  <read_391>
visited read_445
travelled  479 read  <read_445>
visited read_222
travelled  480 read  <read_222>
visited read_294
travelled  481 read  <read_294>
visited read_373
travelled  482 read  <read_373>
visited read_348
travelled  483 read  <read_348>
visited read_360
travelled  484 read  <read_360

In [22]:
def get_read_order(node: Node) -> List[str]:
    order = []
    while node.parent is not None:
        order.append((node.state, node.cost))
        node = node.parent
    order.append((node.state, 0))
    return order[::-1]

In [23]:
order = get_read_order(node)

In [24]:
len(order)

599

In [25]:
node.path_cost

-11990

In [26]:
order = get_read_order(node)

In [32]:
from functools import reduce

In [35]:
total_string_len = reduce(lambda x, y: x + y, map(lambda x: len(x), data.values()))

In [36]:
total_string_len

209193

In [40]:
reduced_len = reduce(lambda x, y: x + y, map(lambda x: x[1], order))

In [41]:
reduced_len

-11990

In [50]:
total_string_len + reduced_len

197203

In [53]:
data['read_68'][-18:], data['read_403'][:18]

('TTTGCAAACCGGAATATA', 'TTTGCAAACCGGAATATA')

In [55]:
data['read_386'], data['read_596']

('GACTTTTGTACTAAGAAAAATATCTTGACTAACCATCTCTTTCTCTCTTCGATGGGTCTCACAAAAATATTAAACCTCTTTCTGATGGAGTCGTAAAAAGTTTTTATCCTTTCTCTCTTCGATAGGTCTCACAAAAATATTAAACCTCTTTCTGATGGTCTCTATAAACGATTGATTTTTCTTACCCTCTAGAGTTTCCTACGGTCGTGGGTCACACATTTTTTTCTAGACACTAAATAAAATAGTAAAATA',
 'TTTTAGTACATTAATATTATATTTTACTATTTTATTTAGTGTCTAGAAAAAAATGTGTGACCCACGACCGTAGGAAACTCTAGAGGGTAAGAAAAATCAATCGTTTATAGAGACCATCAGAAAGAGGTTTAATATTTTTGTGAGACCTATCGAAGAGAGAAAGGATAAAAACTTTTTACGACTCCATCAGAAAGAGGTTTAATATTTTTGTGAGACCCATCGAAGAGAGAAAGAGATGGTTAGTCAAGATATTTTTCTTAGTACAAAAGTCAATGTTTTAAAATATATGGACGAGAATTAATTTGTCTGTATAAAAACTTGTGTGAAATTATGTACTAGAGAAAAAACGTGAGCAGTGTCCCCTACATGGATTTTACAGATCATTTATATTC')

In [43]:
order

[('read_68', 0),
 ('read_403', -18),
 ('read_471', -20),
 ('read_490', -17),
 ('read_188', -19),
 ('read_507', -18),
 ('read_142', -16),
 ('read_378', -19),
 ('read_449', -27),
 ('read_530', -17),
 ('read_176', -27),
 ('read_169', -10),
 ('read_457', -24),
 ('read_27', -30),
 ('read_523', -20),
 ('read_358', -25),
 ('read_129', -10),
 ('read_380', -23),
 ('read_65', -20),
 ('read_183', -14),
 ('read_39', -13),
 ('read_108', -23),
 ('read_30', -30),
 ('read_369', -21),
 ('read_1', -26),
 ('read_228', -18),
 ('read_298', -14),
 ('read_586', -12),
 ('read_277', -17),
 ('read_195', -28),
 ('read_517', -28),
 ('read_589', -19),
 ('read_410', -10),
 ('read_413', -24),
 ('read_71', -12),
 ('read_383', -25),
 ('read_566', -15),
 ('read_0', -24),
 ('read_224', -16),
 ('read_427', -16),
 ('read_258', -25),
 ('read_180', -21),
 ('read_89', -30),
 ('read_85', -28),
 ('read_432', -25),
 ('read_563', -16),
 ('read_96', -24),
 ('read_481', -25),
 ('read_557', -10),
 ('read_388', -19),
 ('read_593', -

In [44]:
len(weights)

599

In [56]:
def recreate_str(order: List[Tuple[str, int]]) -> str:
    final_string: str = data[order[0][0]]
    for i in range(0, len(weights)-1):
        final_string += data[order[i+1][0]][-order[i+1][1]:]
    return final_string

In [60]:
recreate_str(order)[-500:]

'CCTCTTTCTGATGGTCTCTATAAACGATTGATTTTTCTTACCCTCTAGAGTTTCCTACGGTCGTGGGTCACACATTTTTTTCTAGACACTAAATAAAATAGTAAAATATTTTAGTACATTAATATTATATTTTACTATTTTATTTAGTGTCTAGAAAAAAATGTGTGACCCACGACCGTAGGAAACTCTAGAGGGTAAGAAAAATCAATCGTTTATAGAGACCATCAGAAAGAGGTTTAATATTTTTGTGAGACCTATCGAAGAGAGAAAGGATAAAAACTTTTTACGACTCCATCAGAAAGAGGTTTAATATTTTTGTGAGACCCATCGAAGAGAGAAAGAGATGGTTAGTCAAGATATTTTTCTTAGTACAAAAGTCAATGTTTTAAAATATATGGACGAGAATTAATTTGTCTGTATAAAAACTTGTGTGAAATTATGTACTAGAGAAAAAACGTGAGCAGTGTCCCCTACATGGATTTTACAGATCATTTATATTC'

In [49]:
# theoretical length found
total_string_len + reduced_len == len(final_string)

197203

In [ ]:
def exp_schedule(k=20, lam=0.005, limit=100):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)


def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice


def simulated_annealing_full(problem, schedule=exp_schedule()):
    """ This version returns all the states encountered in reaching 
    the goal state."""
    states = []
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        states.append(current.state)
        T = schedule(t)
        if T == 0:
            return states
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice